In [21]:
import loaders.arxiv as arxiv
# import Client, Search
client = arxiv.Client(page_size=2000, )

In [22]:
from urllib.parse import urlencode, quote

query = 'influence functions'
# url = "https://export.arxiv.org/api/query?{}"
# url_params = {
#     'search_query' : q
# }

# urlparams = urlencode(url_params, quote_via=quote)
# url.format(urlparams)
# urllib.parse.(q)

In [23]:
from typing import List
## restricting my search to category computer science
# query = 'cat:Computer Science(cs) AND ti:influence functions'
search = arxiv.Search(query=query, max_results=100, sort_by=arxiv.SortCriterion.SubmittedDate)
results = client.results(search=search)

In [24]:
res = [r for r in results]

In [25]:
for r in res[-5:]:
    print(r.title, r.primary_category, r.published)

The transcritical Bogdanov Takens bifurcation with boundary due to the risk perception on a recruitment epidemiological model math.DS 2024-03-21 05:42:42+00:00
Evolving Benchmark Functions to Compare Evolutionary Algorithms via Genetic Programming cs.NE 2024-03-21 05:42:17+00:00
Genetic Programming for Explainable Manifold Learning cs.NE 2024-03-21 05:17:22+00:00
Temperature dependent conductivity, dielectric relaxation, electrical modulus and impedance spectroscopy of Ni substituted Na$_{3+2x}$Zr$_{2-x}$Ni$_{x}$Si$_2$PO$_{\rm 12}$ cond-mat.mtrl-sci 2024-03-21 05:12:02+00:00
3D Object Detection from Point Cloud via Voting Step Diffusion cs.CV 2024-03-21 05:04:52+00:00


IndexError: list index out of range